<a href="https://colab.research.google.com/github/sera0911/asia_ai_study/blob/main/MachinLearning/ML_%EA%B3%BC%EC%A0%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

uriage.csv - 매출 이력  
kokyaku_daicho.xlsx - 대리점 관리 고객 정보
1. 데이터로드

2. 데이터 오류 체크   
상품명 item_name에 공백, 대소문자 오류, 상품 가격 item_price에 결측치 NaN

3. 데이터에 오류가 있는 상태로 집계 수행, 결과 확인

4. 상품명의 오류 수정
item_price의 결측치는 동일 상품의 단가를 이용해서 수정

5. 금액의 결측치 수정
각 상품의 금액이 정상적으로 수정됐는지 확인

6. 고객 이름의 오류 수정 (공백제거)

7. 날짜 오류 수정

8. 고객 이름을 키로 두 개의 데이터 조인 (매출 이력과 고객 정보 결합)

9. 정제한 데이터를 덤프 (저장, csv)

10. 9단계에서 저장된 파일을 로드 후 데이터 집계 (item별, price별, 고객별)

In [3]:
#1. 데이터로드
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

uriage = pd.read_csv('uriage.csv')
kokyaku = pd.read_excel('kokyaku_daicho.xlsx')

In [4]:
#2. 데이터 오류 체크
#상품명 item_name에 공백, 대소문자 오류, 상품 가격 item_price에 결측치 NaN

uriage.info()
uriage.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2999 entries, 0 to 2998
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   purchase_date  2999 non-null   object 
 1   item_name      2999 non-null   object 
 2   item_price     2612 non-null   float64
 3   customer_name  2999 non-null   object 
dtypes: float64(1), object(3)
memory usage: 93.8+ KB


,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현


In [5]:
#3. 데이터에 오류가 있는 상태로 집계 수행, 결과 확인
uriage['purchase_date'] = pd.to_datetime(uriage['purchase_date'])
uriage['purchase_month'] = uriage['purchase_date'].dt.strftime("%Y%m") #연월을 문자열로 출력
df = uriage.pivot_table(index='purchase_month', columns='item_name', aggfunc='size', fill_value=0)
df

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,상 품T,상 품V,상 품 S,상 품 a,상 품 q,상 품 s,상 품A,상 품C,상 품D,상 품E,상 품F,상 품G,상 품H,상 품I,상 품K,상 품M,상 품N,상 품O,상 품P,상 품T,상 품U,상 품V,상 품X,상 품Y,상 품s,상품 A,상품 B,상품 E,상품 F,상품 H,...,상품D,상품E,상품F,상품G,상품H,상품I,상품J,상품K,상품L,상품M,상품N,상품O,상품P,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z,상품a,상품c,상품d,상품e,상품g,상품i,상품j,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201901,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,0,0,2,0,1,0,0,0,0,0,...,17,18,15,11,15,17,17,19,18,18,15,21,15,17,21,18,16,7,21,13,12,10,0,0,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,...,20,16,14,13,17,11,13,16,11,15,19,18,19,22,21,21,22,19,21,24,14,11,1,0,0,0,0,0,0,1,0,0,0,0,0,1,1,1,0,0
201903,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,16,8,26,14,18,12,15,14,20,21,13,11,20,23,16,20,12,23,17,16,21,16,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,1,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,3,0,0,0,0,2,0,...,20,17,15,13,11,18,13,14,15,11,20,14,15,19,20,15,15,11,14,13,16,17,0,1,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,1,0,0,0,0,0,0,0,0,2,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,...,14,19,17,23,15,16,10,16,12,17,19,18,20,12,22,16,15,16,8,20,16,19,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,...,19,13,18,13,12,18,22,15,15,17,16,14,17,15,16,21,12,17,19,16,14,13,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,...,17,10,17,19,18,18,23,12,17,11,15,19,26,14,17,22,20,13,26,16,18,12,0,0,0,0,0,0,1,0,0,0,1,0,2,0,0,0,0,0


In [6]:
#4. 상품명의 오류 수정 item_price의 결측치는 동일 상품의 단가를 이용해서 수정

uriage['item_name']=uriage['item_name'].str.upper() #소문자도 있기에 모두 대문자로 수정
uriage['item_name']=uriage['item_name'].str.replace(" ", '')  #상품명에 빈칸이 있는 경우 빈칸없이 변경
uriage.sort_values(by=['item_name'], ascending=True)  #상품명으로 정렬

,purchase_date,item_name,item_price,customer_name,purchase_month
0,2019-06-13 18:02:00,상품A,100.0,김가온,201906
1748,2019-05-19 20:22:00,상품A,100.0,김시훈,201905
223,2019-06-25 08:13:00,상품A,100.0,김유진,201906
1742,2019-06-13 16:03:00,상품A,100.0,김건희,201906
1738,2019-02-10 00:28:00,상품A,100.0,김하랑,201902
...,...,...,...,...,...
2880,2019-04-22 00:36:00,상품Y,NaN,김동욱,201904
2881,2019-04-30 14:21:00,상품Y,NaN,김하준,201904
1525,2019-01-24 10:27:00,상품Y,2500.0,김범준,201901
1361,2019-05-28 13:45:00,상품Y,2500.0,김수현,201905


In [7]:
#변경이 잘 됐는지 확인
print(pd.unique(uriage["item_name"]))
print(len(pd.unique(uriage["item_name"])))

['상품A' '상품S' '상품Z' '상품V' '상품O' '상품U' '상품L' '상품C' '상품I' '상품R' '상품X' '상품G'
 '상품P' '상품Q' '상품Y' '상품N' '상품W' '상품E' '상품K' '상품B' '상품F' '상품D' '상품M' '상품H'
 '상품T' '상품J']
26


In [8]:
nul = uriage['item_price'].isnull() 
nul

0       False
1        True
2        True
3       False
4        True
        ...  
2994    False
2995    False
2996     True
2997     True
2998    False
Name: item_price, Length: 2999, dtype: bool

In [9]:
#결측치에 동일한 상품가격이 채워지게 넣어줌
for nan in list(uriage.loc[nul, 'item_name'].unique()):
  price = uriage.loc[(~nul)&(uriage['item_name']==nan), 'item_price'].max()
  uriage['item_price'].loc[(nul)&(uriage['item_name']==nan)]=price 

In [10]:
#5. 금액의 결측치 수정 각 상품의 금액이 정상적으로 수정됐는지 확인
pd.unique(uriage['item_price'].isnull())

array([False])

In [11]:
#6. 상품명의 오류 수정 item_price의 결측치는 동일 상품의 단가를 이용해서 수정

kokyaku['고객이름']=kokyaku['고객이름'].str.replace(" ", "") #이름 사이 빈칸을 수정

kokyaku['고객이름'].head()

0    김현성
1    김도윤
2    김지한
3    김하윤
4    김시온
Name: 고객이름, dtype: object

파이썬 데이터는 정수 시작값이 0, 1900년은 평년이지만 엑셀에서는 1900/2/29를 유효 날짜로 계산  
엑셀 데이터는 정수 시작값이 1, 1900년은 평년이어서 100/2/29날이 없다  
엑셀 날짜 형식의 숫자는 파이썬과 엑셀이 어긋나기 때문에 엑셀 숫자값에서 2를 빼서 처리한다  

In [12]:
#7. 날짜 오류 수정

day = kokyaku['등록일'].astype('str').str.isdigit()
day.sum()

22

In [13]:
#to_timedelta = 기간, unit='D' =기간의 단위
fromday = pd.to_timedelta(kokyaku.loc[day, '등록일']
                          .astype('float')-2, unit='D')+pd.to_datetime('1900/01/01')
fromday

1     2017-02-16
3     2017-05-17
4     2018-01-27
21    2017-07-04
27    2017-06-15
47    2017-01-06
49    2017-07-13
53    2017-04-08
76    2018-03-29
80    2018-01-10
99    2017-05-30
114   2018-06-03
118   2018-01-29
122   2018-04-16
139   2017-05-25
143   2017-03-24
155   2017-01-19
172   2018-03-22
179   2017-01-08
183   2017-07-24
186   2018-07-13
192   2018-06-08
Name: 등록일, dtype: datetime64[ns]

In [14]:
fromstr = pd.to_datetime(kokyaku.loc[~day, '등록일'])
fromstr  #정상적날짜

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 등록일, Length: 178, dtype: datetime64[ns]

In [15]:
kokyaku['등록일'] = pd.concat([fromday, fromstr])
kokyaku

,고객이름,지역,등록일
0,김현성,H시,2018-01-04
1,김도윤,E시,2017-02-16
2,김지한,A시,2018-01-07
3,김하윤,F시,2017-05-17
4,김시온,E시,2018-01-27
...,...,...,...
195,김재희,G시,2017-06-20
196,김도영,E시,2018-06-20
197,김이안,F시,2017-04-29
198,김시현,H시,2019-04-19


In [16]:
kokyaku['등록연월'] = kokyaku['등록일'].dt.strftime('%Y%m')

rslt = kokyaku.groupby("등록연월").count()["고객이름"]
print(rslt)
print(len(kokyaku))


등록연월
201701    15
201702    11
201703    14
201704    15
201705    14
201706    13
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 고객이름, dtype: int64
200


In [17]:
#8. 고객 이름을 키로 두 개의 데이터 조인 (매출 이력과 고객 정보 결합)

df = pd.merge(uriage, kokyaku, left_on='customer_name', right_on='고객이름', how='left')
df = df.drop("customer_name", axis=1)
df

,purchase_date,item_name,item_price,purchase_month,고객이름,지역,등록일,등록연월
0,2019-06-13 18:02:00,상품A,100.0,201906,김가온,C시,2017-01-26,201701
1,2019-07-13 13:05:00,상품S,1900.0,201907,김우찬,C시,2018-04-07,201804
2,2019-05-11 19:42:00,상품A,100.0,201905,김유찬,A시,2018-06-19,201806
3,2019-02-12 23:40:00,상품Z,2600.0,201902,김재현,D시,2018-07-22,201807
4,2019-04-22 03:09:00,상품A,100.0,201904,김강현,D시,2017-06-07,201706
...,...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,상품Y,2500.0,201902,김정민,B시,2017-07-01,201707
2995,2019-06-22 04:03:00,상품M,1300.0,201906,김재원,E시,2018-03-31,201803
2996,2019-03-29 11:14:00,상품Q,1700.0,201903,김지율,B시,2017-03-15,201703
2997,2019-07-14 12:56:00,상품H,800.0,201907,김승주,E시,2018-07-13,201807


In [18]:
#9. 정제한 데이터를 덤프 (저장, csv)

df = df[['purchase_date', 'purchase_month', 'item_name', 'item_price', '고객이름', '지역', '등록일']]
df

,purchase_date,purchase_month,item_name,item_price,고객이름,지역,등록일
0,2019-06-13 18:02:00,201906,상품A,100.0,김가온,C시,2017-01-26
1,2019-07-13 13:05:00,201907,상품S,1900.0,김우찬,C시,2018-04-07
2,2019-05-11 19:42:00,201905,상품A,100.0,김유찬,A시,2018-06-19
3,2019-02-12 23:40:00,201902,상품Z,2600.0,김재현,D시,2018-07-22
4,2019-04-22 03:09:00,201904,상품A,100.0,김강현,D시,2017-06-07
...,...,...,...,...,...,...,...
2994,2019-02-15 02:56:00,201902,상품Y,2500.0,김정민,B시,2017-07-01
2995,2019-06-22 04:03:00,201906,상품M,1300.0,김재원,E시,2018-03-31
2996,2019-03-29 11:14:00,201903,상품Q,1700.0,김지율,B시,2017-03-15
2997,2019-07-14 12:56:00,201907,상품H,800.0,김승주,E시,2018-07-13


In [19]:
df.to_csv('final.csv', index=False)  #저장하기

In [20]:
df = pd.read_csv('final.csv')
df.head()

,purchase_date,purchase_month,item_name,item_price,고객이름,지역,등록일
0,2019-06-13 18:02:00,201906,상품A,100.0,김가온,C시,2017-01-26 00:00:00
1,2019-07-13 13:05:00,201907,상품S,1900.0,김우찬,C시,2018-04-07 00:00:00
2,2019-05-11 19:42:00,201905,상품A,100.0,김유찬,A시,2018-06-19 00:00:00
3,2019-02-12 23:40:00,201902,상품Z,2600.0,김재현,D시,2018-07-22 00:00:00
4,2019-04-22 03:09:00,201904,상품A,100.0,김강현,D시,2017-06-07 00:00:00


In [22]:
#10. 9단계에서 저장된 파일을 로드 후 데이터 집계 (item별, 고객별,  지역별)

Item = df.pivot_table(index="purchase_month", columns="item_name", values="item_price", aggfunc="sum", fill_value=0)
Item


item_name,상품A,상품B,상품C,상품D,상품E,상품F,상품G,상품H,상품I,상품J,상품K,상품L,상품M,상품N,상품O,상품P,상품Q,상품R,상품S,상품T,상품U,상품V,상품W,상품X,상품Y,상품Z
purchase_month,,,,,,,,,,,,,,,,,,,,,,,,,,
201901,1800,2600,5700,6800,9000,9000,7700,12800,16200,17000,22000,22800,24700,22400,36000,25600,28900,37800,38000,34000,14700,48400,29900,33600,25000,0
201902,1900,2800,7800,8400,8000,8400,9800,13600,10800,14000,17600,13200,19500,28000,28500,30400,37400,39600,41800,46000,39900,48400,55200,38400,27500,2600
201903,1700,4200,6000,6800,4500,16200,9800,14400,10800,16000,15400,24000,28600,18200,16500,33600,39100,28800,38000,24000,48300,39600,36800,50400,40000,0
201904,1700,3800,7200,8000,9000,10200,9800,8800,16200,13000,15400,18000,14300,28000,22500,24000,34000,36000,30400,32000,23100,33000,32200,38400,50000,0
201905,2400,2800,4800,5600,9500,10800,16100,12000,14400,11000,19800,15600,23400,26600,27000,32000,22100,39600,34200,32000,33600,19800,48300,38400,50000,0
201906,2400,2400,3300,7600,6500,10800,10500,10400,17100,22000,16500,18000,22100,22400,22500,28800,25500,28800,39900,24000,37800,44000,39100,36000,32500,0
201907,2000,4000,5100,6800,6000,10200,13300,15200,17100,23000,13200,20400,14300,21000,33000,41600,25500,34200,43700,42000,27300,61600,36800,43200,30000,0


In [23]:
Customer = df.pivot_table(index="purchase_month", columns="고객이름", aggfunc="size", fill_value=0)
Customer

고객이름,김가온,김강민,김강현,김건우,김건희,김경민,김규민,김규현,김다온,김대현,김도경,김도영,김도원,김도윤,김도율,김도하,김도현,김도훈,김동건,김동우,김동욱,김동하,김동현,김라온,김민건,김민규,김민기,김민서,김민석,김민수,김민우,김민재,김민준,김민찬,김민혁,김민호,김범준,김상민,김상현,김서윤,...,김지혁,김지호,김지환,김지후,김지훈,김진우,김찬영,김찬우,김태경,김태민,김태양,김태영,김태우,김태윤,김태인,김태준,김태현,김태훈,김하람,김하랑,김하민,김하윤,김하율,김하준,김하진,김한결,김현민,김현서,김현석,김현성,김현수,김현승,김현우,김현준,김현진,김호준,정도형,정영훈,정우석,정준기
purchase_month,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
201901,1,2,1,2,5,3,1,1,1,5,4,0,0,2,2,2,0,3,2,2,2,1,3,3,3,1,3,1,4,2,0,3,3,0,4,4,5,1,2,2,...,3,5,2,2,2,1,6,5,1,2,1,2,3,4,0,0,0,1,1,2,3,3,2,1,7,1,0,1,3,2,3,2,4,2,1,1,4,4,4,3
201902,3,1,4,3,3,5,2,3,1,5,1,6,4,4,1,2,4,3,1,2,0,1,1,3,1,1,3,2,2,3,3,3,9,0,5,2,2,0,1,2,...,3,0,1,1,1,1,1,3,4,1,1,0,1,2,2,0,0,1,2,3,3,2,1,0,2,0,4,4,5,3,5,3,7,0,0,1,2,0,2,2
201903,4,0,3,1,1,2,2,1,1,5,3,0,3,3,0,2,4,2,5,1,1,0,3,1,2,1,5,5,1,0,1,5,1,1,1,1,3,3,0,6,...,4,4,2,1,1,3,0,3,2,2,3,4,3,2,4,2,4,3,1,2,2,1,2,1,1,4,0,5,0,2,3,2,1,2,1,2,1,2,3,4
201904,1,0,3,0,2,1,2,6,2,4,1,3,1,3,1,4,2,2,1,4,4,1,1,3,2,2,4,2,0,1,2,1,0,2,5,4,2,3,2,2,...,1,4,3,1,0,4,1,1,1,5,4,1,0,0,2,1,3,5,0,5,3,4,4,4,1,0,1,4,4,3,0,4,3,1,1,2,2,4,3,1
201905,0,0,2,2,1,4,6,3,3,1,3,2,3,2,2,1,1,1,2,2,0,2,4,4,0,1,2,2,2,1,3,1,3,5,2,1,2,1,2,2,...,3,2,6,3,4,0,0,2,2,4,2,5,1,0,3,1,1,1,1,5,4,4,2,1,1,2,2,2,0,2,2,2,1,4,3,0,0,0,1,2
201906,5,0,3,0,2,1,6,1,1,1,2,2,5,4,2,1,1,2,1,3,1,2,2,3,3,4,3,3,6,2,1,2,1,2,0,2,1,3,4,4,...,0,1,3,1,1,2,2,1,4,4,1,3,2,3,2,3,1,2,2,0,2,3,2,3,0,1,1,1,1,4,2,4,1,0,4,2,2,0,0,2
201907,3,1,3,2,2,1,4,1,4,3,4,2,1,1,3,2,3,1,1,3,5,2,1,2,2,1,1,2,2,1,5,0,3,4,2,2,5,4,4,2,...,2,0,1,2,5,2,2,4,6,1,1,5,2,2,4,1,2,2,2,2,2,2,2,4,1,0,1,3,2,4,2,1,0,4,0,2,6,1,2,3


In [24]:
Region = df.pivot_table(index="purchase_month", columns="지역", aggfunc="size", fill_value=0)
Region


지역,A시,B시,C시,D시,E시,F시,G시,H시
purchase_month,,,,,,,,
201901,59,55,72,34,49,57,49,42
201902,71,46,65,48,61,52,43,63
201903,64,52,57,43,52,59,51,59
201904,64,48,54,45,48,58,40,52
201905,57,52,68,48,59,65,35,43
201906,53,47,61,30,51,51,58,58
201907,76,53,61,42,54,64,47,54
